In [2]:

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
from autorocks.envs.synthetic.funcs.ackley import Ackley6DParametersSpace
from botorch.utils.transforms import normalize, standardize
from autorocks.dir_struct import LocalResultDir
from autorocks.data.dataset import BOSystemDataset

sys_param = Ackley6DParametersSpace()
dataset = BOSystemDataset(
    historic_data_path=LocalResultDir / "synthetic/target/Akcley/6_params/100_iter",
    parameters_name=["x1", "x2", "x3", "x4", "x5", "x6"],
    objectives_name=["target"],
    transform=lambda X: normalize(X, bounds=sys_param.bounds().T),
    target_transform=lambda X: -X,
    filter_for_specific_models={"StaticBoGraphStandardizeHigh"},
)

In [594]:
import torch.utils.data

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

In [1]:
from torch.utils.data import DataLoader

num_of_samples = 9
train_dataloader = DataLoader(train_set, batch_size=num_of_samples, shuffle=True)
data_loader_iter = iter(train_dataloader)
test_x, test_y = test_set[:]
from sklearn.preprocessing import StandardScaler

In [615]:
from autorocks.optimizer.bograph.dag_dao.model_nodes.node_singlegp import SingleTaskGPModelNode
import gpytorch.means as means

train_x, train_y = next(data_loader_iter)
# y_standardize = StandardScaler().fit(train_y)
# train_y = torch.tensor(y_standardize.transform(train_y))

node_model = SingleTaskGPModelNode(train_x=train_x, train_y=train_y, mean_module=means.ZeroMean())

# test_y = torch.tensor(y_standardize.transform(test_y)).squeeze()
test_y = test_y.squeeze()

# MSE and RMSE calculation

In [616]:
with torch.no_grad():
    posterior = node_model.posterior(X=test_x)
    predicted_mean = posterior.mean.squeeze()

    std = torch.sqrt(posterior.variance.squeeze()) * 2
    lower, upper = predicted_mean - std, predicted_mean + std
    lower, upper = lower.squeeze().cpu().numpy(), upper.squeeze().cpu().numpy()

    mse = torch.mean(torch.abs(predicted_mean - test_y))
    rmse = torch.sqrt(torch.mean(torch.pow(predicted_mean - test_y, 2)))

print(f"RMSE = {rmse}, MSE = {mse}")

# Predicted outcome vs actual outcome

In [617]:
from matplotlib import pyplot as plt

f, ax = plt.subplots(1, 1, figsize=(16, 9))
mean_sorted_args = torch.argsort(test_y.squeeze())
ax.plot(test_y, test_y, color="r", label="Ideal outcome")
ax.plot(test_y[mean_sorted_args], predicted_mean[mean_sorted_args], label="Prediction to outcome")
# Shade between the lower and upper confidence bounds
ax.fill_between(test_y[mean_sorted_args], y1=lower[mean_sorted_args], y2=upper[mean_sorted_args], alpha=0.5, label="CI")
ax.set(xlabel="Actual outcome", ylabel="Predicted outcome")
ax.legend()

# Model fit of the observed data point

In [618]:
from sklearn.decomposition import PCA
import numpy as np

# Visualize what has been observed
pca = PCA(n_components=1)
decomposed_x = pca.fit_transform(train_x, train_y)
sorted_x = np.argsort(decomposed_x.squeeze())
f, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.plot(decomposed_x[sorted_x], train_y[sorted_x], "k*", label="Observed data points")
with torch.no_grad():
    observed_posterior = node_model.posterior(X=train_x)
    predicted_mean = observed_posterior.mean.squeeze()
    std = torch.sqrt(observed_posterior.variance.squeeze()) * 2
    lower, upper = predicted_mean - std, predicted_mean + std
    lower, upper = lower.squeeze().cpu().numpy(), upper.squeeze().cpu().numpy()

ax.plot(decomposed_x[sorted_x], predicted_mean[sorted_x], color="r", label="Model fit")
ax.fill_between(decomposed_x[sorted_x].squeeze(), y1=lower[sorted_x], y2=upper[sorted_x], alpha=0.5, label="CI")

ax.set(xlabel="Compressed X", ylabel="System objective", title="Observed data")
ax.legend()

# Predicted outcome vs actual outcome against various values of X


In [619]:
# Visualize what has been observed
pca = PCA(n_components=1)
decomposed_x = pca.fit_transform(test_x, test_y)
sorted_x = np.argsort(decomposed_x.squeeze())


f, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.plot(decomposed_x[sorted_x], test_y[sorted_x], color="g", label="Actual data")
with torch.no_grad():
    observed_posterior = node_model.posterior(X=test_x)
    predicted_mean = observed_posterior.mean.squeeze()
    std = torch.sqrt(observed_posterior.variance.squeeze()) * 2
    lower, upper = predicted_mean - std, predicted_mean + std
    lower, upper = lower.squeeze().cpu().numpy(), upper.squeeze().cpu().numpy()

ax.plot(decomposed_x[sorted_x], predicted_mean[sorted_x], color="r", label="Model prediction")
ax.fill_between(decomposed_x[sorted_x].squeeze(), y1=lower[sorted_x], y2=upper[sorted_x], alpha=0.5, label="CI")

ax.set(xlabel="Compressed X", ylabel="System objective", title="Model prediction vs test data")
ax.legend()

# Acquisition function exploring

[ ] Explore how the ACQF will choose action given what it observed:
[ ] run the optimizer and choose the value being chosen next, and the value of the acqf
[ ] plot both against the PCAed plot

In [623]:
from botorch.sampling import SobolQMCNormalSampler
from botorch.optim import optimize_acqf
import autorocks.optimizer.acqf as acqf_wrapper


acqf_cfg = acqf_wrapper.qUpperConfidenceBoundWrapper(beta=0.2, sampler=SobolQMCNormalSampler(num_samples=1024))
acqf = acqf_cfg.build(
    model=node_model,
    observed_x=train_x,
    observed_y=train_y,
)

candidates, acqf_val = optimize_acqf(
    acq_function=acqf,
    bounds=torch.stack(
        [
            torch.zeros(sys_param.dimensions, dtype=torch.double),
            torch.ones(sys_param.dimensions, dtype=torch.double),
        ]
    ),
    q=1,
    num_restarts=12,
    raw_samples=1024,
)

print(f"Candidate: {candidates}, ACQF_Value: {acqf_val}")

from sklearn.decomposition import PCA
import numpy as np

# Visualize what has been observed
pca = PCA(n_components=1)
decomposed_x = pca.fit_transform(train_x, train_y)
sorted_x = np.argsort(decomposed_x.squeeze())
f, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.plot(decomposed_x[sorted_x], train_y[sorted_x], "k*", label="Observed data points")
with torch.no_grad():
    observed_posterior = node_model.posterior(X=train_x)
    predicted_mean = observed_posterior.mean.squeeze()
    std = torch.sqrt(observed_posterior.variance.squeeze()) * 2
    lower, upper = predicted_mean - std, predicted_mean + std
    lower, upper = lower.squeeze().cpu().numpy(), upper.squeeze().cpu().numpy()

ax.plot(decomposed_x[sorted_x], predicted_mean[sorted_x], color="r", label="Model fit")
ax.fill_between(decomposed_x[sorted_x].squeeze(), y1=lower[sorted_x], y2=upper[sorted_x], alpha=0.5, label="CI")

compressed_candidates = pca.transform(candidates)
ax.axvline(x=compressed_candidates, label="ACQF Chosen point", color="g")

ax.set(xlabel="Compressed X", ylabel="System objective", title="Observed data")
ax.legend()

In [559]:
acqf.forward(torch.tensor([[0.4820, 0.5089, 0.5125, 0.5475, 0.4980, 0.4795]]).unsqueeze(0))

# Probabilistic DAG prediction

[ ] Pass a static graph, create probabilistic DAG
[ ] Show prediction plot for each node in the graph
[ ] Show RMSE of each node in the graph, then RMSE of the main graph

In [342]:
# TODO: Given a full graph, create the BoGraph Probabilistic DAG

# BOGraph Full loop

[ ] Pass in the dataset (pandas) and create BoGraph full loop